In [ ]:
from hope.dataset.dataset import CMPFacadeDataset
from hope.models.hope import HopeModel

In [ ]:
# model = HopeModel(512)

In [ ]:
dataset = CMPFacadeDataset('/home/penguin/SSIW/data/base')

In [ ]:
import torch.nn as nn

In [ ]:
for e in dataset:
    image, mask = e
    break

In [ ]:
# model = model.to('cuda')

In [ ]:
# logit, mask = model(image.to('cuda'), mask)

In [ ]:
from hope.utils.collator import MyCollator

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
coll = MyCollator()

In [ ]:
dataloader = DataLoader(dataset, 2, collate_fn=coll)

In [ ]:
for e in dataloader:
    print(e)

In [ ]:
image.shape

In [ ]:
logit.shape, mask.shape


In [ ]:
import torch

In [ ]:
criteria = torch.nn.CrossEntropyLoss()

In [ ]:
logit = logit.to('cpu')
mask = mask.to('cpu')

In [ ]:
criteria(logit, mask.unsqueeze(0))